# Training on multiple topologies
Eamon Whalen

In [1]:
import sys
import glob
import numpy as np
import pandas as pd
import altair as alt
from sklearn.model_selection import train_test_split

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
trainSets ,valSets ,testSets = {}, {}, {}

doeFiles = np.sort(glob.glob("/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/*.csv"))
for doeFile in doeFiles:
    designName = doeFile.split('/')[-1].split('_N')[0]
    print(f'loading {designName}')
    allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)
    allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
    trainData, valData, testData = partitionGraphList(allGraphs)
    trainSets[designName] = trainData
    valSets[designName] = valData
    testSets[designName] = testData

loading design_5
loading design_6
loading design_7
loading design_8
loading design_9


## 2. Train on each group seperately

In [3]:
resultsList = []
saveDir = './results/topoTest01/'
epochs = 100

In [4]:
for trainName, trainSet in trainSets.items():
    print('training on '+trainName)

    # train
    gcn = FeaStNet()
    history = gcn.trainModel(trainSet, valSets[trainName], epochs=epochs, batch_size=256, flatten=True, logTrans=False, 
                             ssTrans=True, saveDir=saveDir+trainName)

    display(plotHistory(history))

    # test
    print('testing on '+trainName+'\n')
    resultsDict = gcn.testModel(testSets[trainName], level='field')
    resultsDict['Trained on'] = ['test group']*len(resultsDict['mse'])
    resultsDict['Tested on'] = [trainName]*len(resultsDict['mse'])
    results = pivotDict(resultsDict)
    resultsList.extend(results)
        
pd.DataFrame(resultsList)

training on design_5
epoch: 0   trainLoss: 1.0236e+00   valLoss:1.0341e+00  time: 1.30e+00
epoch: 1   trainLoss: 9.1437e-01   valLoss:1.0340e+00  time: 9.70e-01
epoch: 2   trainLoss: 9.1646e-01   valLoss:1.0371e+00  time: 9.55e-01
epoch: 3   trainLoss: 8.6678e-01   valLoss:1.0667e+00  time: 1.05e+00
epoch: 4   trainLoss: 8.4037e-01   valLoss:1.0750e+00  time: 9.49e-01
epoch: 5   trainLoss: 7.9032e-01   valLoss:1.0515e+00  time: 9.45e-01
epoch: 6   trainLoss: 7.0411e-01   valLoss:9.9065e-01  time: 9.48e-01
epoch: 7   trainLoss: 7.5412e-01   valLoss:8.8625e-01  time: 9.38e-01
epoch: 8   trainLoss: 6.4237e-01   valLoss:7.4567e-01  time: 9.32e-01
epoch: 9   trainLoss: 6.4650e-01   valLoss:6.6805e-01  time: 9.36e-01
epoch: 10   trainLoss: 6.4561e-01   valLoss:6.1587e-01  time: 9.42e-01
epoch: 11   trainLoss: 5.7974e-01   valLoss:5.8267e-01  time: 9.39e-01
epoch: 12   trainLoss: 5.6496e-01   valLoss:5.5390e-01  time: 9.53e-01
epoch: 13   trainLoss: 4.9094e-01   valLoss:5.2097e-01  time: 9.26

alt.Chart(...)

testing on design_5

training on design_6
epoch: 0   trainLoss: 1.0307e+00   valLoss:1.1964e+00  time: 9.85e-01
epoch: 1   trainLoss: 9.8989e-01   valLoss:1.1953e+00  time: 9.83e-01
epoch: 2   trainLoss: 9.1991e-01   valLoss:1.2006e+00  time: 9.82e-01
epoch: 3   trainLoss: 8.4868e-01   valLoss:1.2184e+00  time: 9.82e-01
epoch: 4   trainLoss: 8.4191e-01   valLoss:1.2214e+00  time: 9.76e-01
epoch: 5   trainLoss: 8.1276e-01   valLoss:1.2122e+00  time: 9.94e-01
epoch: 6   trainLoss: 8.1783e-01   valLoss:1.1453e+00  time: 9.74e-01
epoch: 7   trainLoss: 7.5898e-01   valLoss:1.0512e+00  time: 9.68e-01
epoch: 8   trainLoss: 7.2891e-01   valLoss:9.0356e-01  time: 9.71e-01
epoch: 9   trainLoss: 6.9601e-01   valLoss:7.9334e-01  time: 9.70e-01
epoch: 10   trainLoss: 6.8612e-01   valLoss:7.3645e-01  time: 9.75e-01
epoch: 11   trainLoss: 6.2659e-01   valLoss:6.9473e-01  time: 9.74e-01
epoch: 12   trainLoss: 5.5066e-01   valLoss:6.5132e-01  time: 9.85e-01
epoch: 13   trainLoss: 5.4008e-01   valLoss:6

alt.Chart(...)

testing on design_6

training on design_7
epoch: 0   trainLoss: 9.8301e-01   valLoss:9.6612e-01  time: 1.02e+00
epoch: 1   trainLoss: 8.4510e-01   valLoss:9.6547e-01  time: 1.01e+00
epoch: 2   trainLoss: 8.3194e-01   valLoss:9.6598e-01  time: 1.01e+00
epoch: 3   trainLoss: 7.8685e-01   valLoss:9.6204e-01  time: 1.00e+00
epoch: 4   trainLoss: 7.1452e-01   valLoss:9.5872e-01  time: 1.01e+00
epoch: 5   trainLoss: 7.0047e-01   valLoss:1.0128e+00  time: 1.01e+00
epoch: 6   trainLoss: 6.6699e-01   valLoss:1.0624e+00  time: 1.01e+00
epoch: 7   trainLoss: 6.8170e-01   valLoss:9.8703e-01  time: 1.01e+00
epoch: 8   trainLoss: 6.2906e-01   valLoss:8.1915e-01  time: 1.02e+00
epoch: 9   trainLoss: 6.0512e-01   valLoss:7.2183e-01  time: 1.02e+00
epoch: 10   trainLoss: 6.1551e-01   valLoss:6.7612e-01  time: 1.02e+00
epoch: 11   trainLoss: 5.8505e-01   valLoss:6.5659e-01  time: 1.01e+00
epoch: 12   trainLoss: 5.5821e-01   valLoss:6.3162e-01  time: 9.86e-01
epoch: 13   trainLoss: 5.5837e-01   valLoss:6

...

alt.Chart(...)

testing on design_8

training on design_9
epoch: 0   trainLoss: 1.1523e+00   valLoss:1.4104e+00  time: 1.05e+00
epoch: 1   trainLoss: 9.9646e-01   valLoss:1.4056e+00  time: 1.14e+00
epoch: 2   trainLoss: 9.6904e-01   valLoss:1.3825e+00  time: 1.07e+00
epoch: 3   trainLoss: 8.5773e-01   valLoss:1.3552e+00  time: 1.06e+00
epoch: 4   trainLoss: 7.9862e-01   valLoss:1.3481e+00  time: 1.04e+00
epoch: 5   trainLoss: 7.7294e-01   valLoss:1.4210e+00  time: 1.04e+00
epoch: 6   trainLoss: 7.0956e-01   valLoss:1.4228e+00  time: 1.04e+00
epoch: 7   trainLoss: 6.6632e-01   valLoss:1.2305e+00  time: 1.05e+00
epoch: 8   trainLoss: 6.1182e-01   valLoss:9.6488e-01  time: 1.05e+00
epoch: 9   trainLoss: 5.6247e-01   valLoss:7.9064e-01  time: 1.05e+00
epoch: 10   trainLoss: 5.4153e-01   valLoss:6.9800e-01  time: 1.05e+00
epoch: 11   trainLoss: 5.0868e-01   valLoss:6.2596e-01  time: 1.06e+00
epoch: 12   trainLoss: 5.0838e-01   valLoss:5.8320e-01  time: 1.04e+00
epoch: 13   trainLoss: 4.3988e-01   valLoss:5

alt.Chart(...)

testing on design_9



,mse,mae,mre,maxAE,mae/peak,maxAE/peak,relEAtPeak,Trained on,Tested on
0,0.000003,0.001280,0.086752,0.004402,0.078154,0.268710,0.194701,test group,design_5
1,0.000006,0.001951,0.113409,0.006593,0.058484,0.197649,0.045118,test group,design_5
2,0.000040,0.005222,0.143329,0.013249,0.091799,0.232906,0.232906,test group,design_5
3,0.000022,0.003561,0.077223,0.010990,0.036738,0.113382,0.072949,test group,design_5
4,0.000007,0.002158,0.142622,0.005293,0.152062,0.372986,0.138145,test group,design_5
...,...,...,...,...,...,...,...,...,...
895,0.000030,0.004380,0.174423,0.012689,0.118010,0.341894,0.009775,test group,design_9
896,0.000118,0.008332,0.130676,0.033631,0.051945,0.209677,0.209677,test group,design_9
897,0.000028,0.004457,0.191953,0.012887,0.134846,0.389866,0.028532,test group,design_9
898,0.000016,0.003403,0.109791,0.008502,0.061839,0.154511,0.010709,test group,design_9


## 3. Train on all groups at once

In [5]:
allTrainData, allValData = [], []
print('training on all groups')
for name, data in trainSets.items():
    allTrainData = allTrainData + data
    allValData = allValData + valSets[name]

gcn = FeaStNet()
history = gcn.trainModel(allTrainData, allValData, epochs=epochs, batch_size=256, flatten=True, logTrans=False, 
                         ssTrans=True, saveDir=saveDir+'allGroups')

display(plotHistory(history))

# test
for testName, testSet in testSets.items():
    print('testing on '+testName+'\n')
    resultsDict = gcn.testModel(testSet, level='field')
    resultsDict['Trained on'] = ['all groups']*len(resultsDict['mse'])
    resultsDict['Tested on'] = [testName]*len(resultsDict['mse'])
    results = pivotDict(resultsDict)
    resultsList.extend(results)
    
pd.DataFrame(resultsList)

training on all groups
epoch: 0   trainLoss: 8.4633e-01   valLoss:1.0282e+00  time: 4.79e+00
epoch: 1   trainLoss: 6.8211e-01   valLoss:8.6518e-01  time: 4.98e+00
epoch: 2   trainLoss: 5.6260e-01   valLoss:5.7501e-01  time: 4.87e+00
epoch: 3   trainLoss: 4.7141e-01   valLoss:4.9659e-01  time: 4.80e+00
epoch: 4   trainLoss: 4.0853e-01   valLoss:4.1569e-01  time: 4.87e+00
epoch: 5   trainLoss: 3.5830e-01   valLoss:3.6907e-01  time: 4.77e+00
epoch: 6   trainLoss: 3.2085e-01   valLoss:3.4650e-01  time: 4.88e+00
epoch: 7   trainLoss: 2.8762e-01   valLoss:3.1609e-01  time: 4.83e+00
epoch: 8   trainLoss: 2.6394e-01   valLoss:2.8572e-01  time: 4.90e+00
epoch: 9   trainLoss: 2.4740e-01   valLoss:2.6557e-01  time: 4.87e+00
epoch: 10   trainLoss: 2.3154e-01   valLoss:2.5822e-01  time: 4.82e+00
epoch: 11   trainLoss: 2.1765e-01   valLoss:2.6059e-01  time: 4.97e+00
epoch: 12   trainLoss: 1.9877e-01   valLoss:2.3386e-01  time: 4.79e+00
epoch: 13   trainLoss: 1.9183e-01   valLoss:2.2199e-01  time: 4.

alt.Chart(...)

testing on design_5

testing on design_6

testing on design_7

testing on design_8

testing on design_9



,mse,mae,mre,maxAE,mae/peak,maxAE/peak,relEAtPeak,Trained on,Tested on
0,0.000003,0.001280,0.086752,0.004402,0.078154,0.268710,0.194701,test group,design_5
1,0.000006,0.001951,0.113409,0.006593,0.058484,0.197649,0.045118,test group,design_5
2,0.000040,0.005222,0.143329,0.013249,0.091799,0.232906,0.232906,test group,design_5
3,0.000022,0.003561,0.077223,0.010990,0.036738,0.113382,0.072949,test group,design_5
4,0.000007,0.002158,0.142622,0.005293,0.152062,0.372986,0.138145,test group,design_5
...,...,...,...,...,...,...,...,...,...
1795,0.000009,0.002448,0.102241,0.006308,0.065952,0.169976,0.007431,all groups,design_9
1796,0.000052,0.005090,0.066974,0.020977,0.031737,0.130782,0.086903,all groups,design_9
1797,0.000029,0.004250,0.149318,0.014302,0.128590,0.432672,0.277234,all groups,design_9
1798,0.000008,0.002106,0.064101,0.007909,0.038274,0.143733,0.000555,all groups,design_9


## 4. Leave one out

In [6]:
checkptFiles = {}
for trainName, trainSet in trainSets.items():
    allTrainData, allValData = [], []
    print('training on all but '+ trainName)
    for name, data in trainSets.items():
        if name != trainName:
            allTrainData = allTrainData + data
            allValData = allValData + valSets[name]

    gcn = FeaStNet()
    history = gcn.trainModel(allTrainData, allValData, epochs=epochs, batch_size=256, flatten=True, logTrans=False, 
                             ssTrans=True, saveDir=saveDir+'allBut_'+ trainName)

    display(plotHistory(history))

    # test
    print('testing on '+trainName+'\n')
    resultsDict = gcn.testModel(testSets[trainName], level='field')
    resultsDict['Trained on'] = ['all groups but test group']*len(resultsDict['mse'])
    resultsDict['Tested on'] = [trainName]*len(resultsDict['mse'])
    results = pivotDict(resultsDict)
    resultsList.extend(results)
    
    # note checkpoint files for use in transfer learning
    checkptFiles[trainName] = gcn.checkptFile
    
pd.DataFrame(resultsList)

training on all but design_5
epoch: 0   trainLoss: 9.1784e-01   valLoss:1.0647e+00  time: 4.08e+00
epoch: 1   trainLoss: 7.7366e-01   valLoss:1.0220e+00  time: 3.95e+00
epoch: 2   trainLoss: 6.6175e-01   valLoss:6.9672e-01  time: 3.92e+00
epoch: 3   trainLoss: 5.6353e-01   valLoss:5.5144e-01  time: 3.95e+00
epoch: 4   trainLoss: 4.8355e-01   valLoss:4.7648e-01  time: 3.93e+00
epoch: 5   trainLoss: 4.2624e-01   valLoss:4.0999e-01  time: 3.91e+00
epoch: 6   trainLoss: 3.7408e-01   valLoss:3.8407e-01  time: 3.88e+00
epoch: 7   trainLoss: 3.4730e-01   valLoss:3.4477e-01  time: 3.83e+00
epoch: 8   trainLoss: 3.2042e-01   valLoss:3.2039e-01  time: 3.84e+00
epoch: 9   trainLoss: 3.0368e-01   valLoss:3.0599e-01  time: 3.94e+00
epoch: 10   trainLoss: 2.8380e-01   valLoss:2.9162e-01  time: 3.92e+00
epoch: 11   trainLoss: 2.6562e-01   valLoss:2.9070e-01  time: 3.89e+00
epoch: 12   trainLoss: 2.5146e-01   valLoss:2.6399e-01  time: 3.94e+00
epoch: 13   trainLoss: 2.3598e-01   valLoss:2.5197e-01  ti

alt.Chart(...)

testing on design_5

training on all but design_6
epoch: 0   trainLoss: 9.5387e-01   valLoss:1.0485e+00  time: 3.97e+00
epoch: 1   trainLoss: 7.9212e-01   valLoss:9.9573e-01  time: 3.87e+00
epoch: 2   trainLoss: 6.7770e-01   valLoss:7.0455e-01  time: 3.87e+00
epoch: 3   trainLoss: 5.7420e-01   valLoss:5.8520e-01  time: 3.89e+00
epoch: 4   trainLoss: 5.0010e-01   valLoss:5.2174e-01  time: 3.88e+00
epoch: 5   trainLoss: 4.4195e-01   valLoss:4.7980e-01  time: 3.86e+00
epoch: 6   trainLoss: 3.9473e-01   valLoss:4.4410e-01  time: 3.85e+00
epoch: 7   trainLoss: 3.5718e-01   valLoss:3.9407e-01  time: 3.84e+00
epoch: 8   trainLoss: 3.2837e-01   valLoss:3.6640e-01  time: 3.83e+00
epoch: 9   trainLoss: 3.0285e-01   valLoss:3.5471e-01  time: 3.80e+00
epoch: 10   trainLoss: 2.8829e-01   valLoss:3.3531e-01  time: 3.90e+00
epoch: 11   trainLoss: 2.6421e-01   valLoss:2.9894e-01  time: 3.91e+00
epoch: 12   trainLoss: 2.4656e-01   valLoss:2.7603e-01  time: 3.84e+00
epoch: 13   trainLoss: 2.3697e-01   v

alt.Chart(...)

testing on design_6

training on all but design_7
epoch: 0   trainLoss: 9.6234e-01   valLoss:1.0740e+00  time: 3.90e+00
epoch: 1   trainLoss: 8.0624e-01   valLoss:9.6652e-01  time: 3.86e+00
epoch: 2   trainLoss: 6.9563e-01   valLoss:7.3238e-01  time: 3.78e+00
epoch: 3   trainLoss: 6.1910e-01   valLoss:6.2569e-01  time: 3.81e+00
epoch: 4   trainLoss: 5.4412e-01   valLoss:5.5480e-01  time: 3.85e+00
epoch: 5   trainLoss: 4.8553e-01   valLoss:5.0377e-01  time: 3.82e+00
epoch: 6   trainLoss: 4.2833e-01   valLoss:4.4681e-01  time: 3.79e+00
epoch: 7   trainLoss: 3.8271e-01   valLoss:4.2956e-01  time: 3.87e+00
epoch: 8   trainLoss: 3.4458e-01   valLoss:3.7443e-01  time: 3.85e+00
epoch: 9   trainLoss: 3.2694e-01   valLoss:3.5077e-01  time: 3.83e+00
epoch: 10   trainLoss: 3.0157e-01   valLoss:3.3318e-01  time: 3.75e+00
epoch: 11   trainLoss: 2.7721e-01   valLoss:3.4559e-01  time: 3.82e+00
epoch: 12   trainLoss: 2.6218e-01   valLoss:3.1749e-01  time: 3.85e+00
epoch: 13   trainLoss: 2.4726e-01   v

...

alt.Chart(...)

testing on design_8

training on all but design_9
epoch: 0   trainLoss: 9.8225e-01   valLoss:1.0393e+00  time: 3.87e+00
epoch: 1   trainLoss: 8.5089e-01   valLoss:1.0459e+00  time: 3.74e+00
epoch: 2   trainLoss: 7.7140e-01   valLoss:7.9480e-01  time: 3.77e+00
epoch: 3   trainLoss: 6.9197e-01   valLoss:6.9371e-01  time: 3.75e+00
epoch: 4   trainLoss: 6.2704e-01   valLoss:6.1659e-01  time: 3.77e+00
epoch: 5   trainLoss: 5.6613e-01   valLoss:5.5288e-01  time: 3.74e+00
epoch: 6   trainLoss: 4.9606e-01   valLoss:4.8788e-01  time: 3.74e+00
epoch: 7   trainLoss: 4.3869e-01   valLoss:4.5372e-01  time: 3.80e+00
epoch: 8   trainLoss: 3.9099e-01   valLoss:3.7902e-01  time: 3.73e+00
epoch: 9   trainLoss: 3.4257e-01   valLoss:3.3812e-01  time: 3.74e+00
epoch: 10   trainLoss: 3.0270e-01   valLoss:2.9270e-01  time: 3.74e+00
epoch: 11   trainLoss: 2.7485e-01   valLoss:2.5635e-01  time: 3.74e+00
epoch: 12   trainLoss: 2.5200e-01   valLoss:2.3446e-01  time: 3.73e+00
epoch: 13   trainLoss: 2.2273e-01   v

alt.Chart(...)

testing on design_9



,mse,mae,mre,maxAE,mae/peak,maxAE/peak,relEAtPeak,Trained on,Tested on
0,0.000003,0.001280,0.086752,0.004402,0.078154,0.268710,0.194701,test group,design_5
1,0.000006,0.001951,0.113409,0.006593,0.058484,0.197649,0.045118,test group,design_5
2,0.000040,0.005222,0.143329,0.013249,0.091799,0.232906,0.232906,test group,design_5
3,0.000022,0.003561,0.077223,0.010990,0.036738,0.113382,0.072949,test group,design_5
4,0.000007,0.002158,0.142622,0.005293,0.152062,0.372986,0.138145,test group,design_5
...,...,...,...,...,...,...,...,...,...
2695,0.000124,0.008834,0.325860,0.028798,0.238018,0.775945,0.465729,all groups but test group,design_9
2696,0.000498,0.019481,0.263836,0.040249,0.121456,0.250939,0.241266,all groups but test group,design_9
2697,0.000013,0.002808,0.114196,0.008695,0.084966,0.263051,0.072333,all groups but test group,design_9
2698,0.000057,0.005923,0.167849,0.017815,0.107652,0.323779,0.143389,all groups but test group,design_9


## 5. Transfer learning

In [7]:
allIds = list(range(len(trainSet)))
_, trainIds = train_test_split(allIds, test_size=0.2, shuffle=True, random_state=1234) # train on only 25%

for trainName, trainSet in trainSets.items():
    # load pre-trained model and train on new data
    print('transfer learning on '+trainName)
    gcn = FeaStNet()
    smallTrainSet = [trainSet[i] for i in trainIds]
    history = gcn.trainModel(smallTrainSet, valSets[trainName], restartFile=checkptFiles[trainName], 
                             epochs=epochs, batch_size=256, saveDir=saveDir+'transferLearn_'+ trainName)

    display(plotHistory(history))

    # test
    print('testing on '+trainName+'\n')
    resultsDict = gcn.testModel(testSets[trainName], level='field')
    resultsDict['Trained on'] = ['transfer learning (20%)']*len(resultsDict['mse'])
    resultsDict['Tested on'] = [trainName]*len(resultsDict['mse'])
    results = pivotDict(resultsDict)
    resultsList.extend(results)
        
pd.DataFrame(resultsList)

transfer learning on design_5
loading restart file
epoch: 0   trainLoss: 9.9026e-01   valLoss:2.6003e-01  time: 6.96e-01
epoch: 1   trainLoss: 7.7292e-01   valLoss:2.5094e-01  time: 6.95e-01
epoch: 2   trainLoss: 6.3027e-01   valLoss:2.6007e-01  time: 7.00e-01
epoch: 3   trainLoss: 5.3201e-01   valLoss:2.7360e-01  time: 6.98e-01
epoch: 4   trainLoss: 4.6074e-01   valLoss:2.8280e-01  time: 6.98e-01
epoch: 5   trainLoss: 4.0634e-01   valLoss:2.8185e-01  time: 6.92e-01
epoch: 6   trainLoss: 3.6268e-01   valLoss:2.7179e-01  time: 6.98e-01
epoch: 7   trainLoss: 3.2590e-01   valLoss:2.5740e-01  time: 6.94e-01
epoch: 8   trainLoss: 2.9403e-01   valLoss:2.4222e-01  time: 6.92e-01
epoch: 9   trainLoss: 2.6622e-01   valLoss:2.2990e-01  time: 7.02e-01
epoch: 10   trainLoss: 2.4238e-01   valLoss:2.2048e-01  time: 6.96e-01
epoch: 11   trainLoss: 2.2200e-01   valLoss:2.1334e-01  time: 7.34e-01
epoch: 12   trainLoss: 2.0427e-01   valLoss:2.0694e-01  time: 6.98e-01
epoch: 13   trainLoss: 1.8826e-01   

alt.Chart(...)

testing on design_5

transfer learning on design_6
loading restart file
epoch: 0   trainLoss: 3.9946e-01   valLoss:1.0957e-01  time: 7.17e-01
epoch: 1   trainLoss: 2.3110e-01   valLoss:1.2905e-01  time: 7.38e-01
epoch: 2   trainLoss: 1.5713e-01   valLoss:1.4331e-01  time: 7.38e-01
epoch: 3   trainLoss: 1.2741e-01   valLoss:1.5090e-01  time: 7.35e-01
epoch: 4   trainLoss: 1.1326e-01   valLoss:1.5463e-01  time: 7.26e-01
epoch: 5   trainLoss: 1.0108e-01   valLoss:1.5500e-01  time: 7.30e-01
epoch: 6   trainLoss: 8.7661e-02   valLoss:1.5166e-01  time: 7.31e-01
epoch: 7   trainLoss: 7.4430e-02   valLoss:1.4572e-01  time: 7.32e-01
epoch: 8   trainLoss: 6.3555e-02   valLoss:1.3958e-01  time: 7.24e-01
epoch: 9   trainLoss: 5.5732e-02   valLoss:1.3414e-01  time: 7.25e-01
epoch: 10   trainLoss: 5.0135e-02   valLoss:1.2827e-01  time: 7.26e-01
epoch: 11   trainLoss: 4.5897e-02   valLoss:1.2222e-01  time: 7.32e-01
epoch: 12   trainLoss: 4.2566e-02   valLoss:1.1672e-01  time: 7.43e-01
epoch: 13   tra

alt.Chart(...)

testing on design_6

transfer learning on design_7
loading restart file
epoch: 0   trainLoss: 1.3752e-01   valLoss:1.1919e-01  time: 7.36e-01
epoch: 1   trainLoss: 1.0685e-01   valLoss:8.5733e-02  time: 7.59e-01
epoch: 2   trainLoss: 7.3677e-02   valLoss:8.2287e-02  time: 7.49e-01
epoch: 3   trainLoss: 6.1455e-02   valLoss:8.4801e-02  time: 7.41e-01
epoch: 4   trainLoss: 5.5787e-02   valLoss:8.0559e-02  time: 7.38e-01
epoch: 5   trainLoss: 4.8228e-02   valLoss:7.8461e-02  time: 7.39e-01
epoch: 6   trainLoss: 4.3603e-02   valLoss:7.9658e-02  time: 7.38e-01
epoch: 7   trainLoss: 4.1204e-02   valLoss:8.0573e-02  time: 7.38e-01
epoch: 8   trainLoss: 3.8180e-02   valLoss:7.9460e-02  time: 7.34e-01
epoch: 9   trainLoss: 3.4376e-02   valLoss:7.6955e-02  time: 7.39e-01
epoch: 10   trainLoss: 3.0800e-02   valLoss:7.4908e-02  time: 7.40e-01
epoch: 11   trainLoss: 2.8131e-02   valLoss:7.4834e-02  time: 7.36e-01
epoch: 12   trainLoss: 2.6442e-02   valLoss:7.5471e-02  time: 7.34e-01
epoch: 13   tra

...

alt.Chart(...)

testing on design_8

transfer learning on design_9
loading restart file
epoch: 0   trainLoss: 1.8422e+00   valLoss:9.8440e-01  time: 7.46e-01
epoch: 1   trainLoss: 1.3906e+00   valLoss:1.0150e+00  time: 7.45e-01
epoch: 2   trainLoss: 1.1378e+00   valLoss:1.0223e+00  time: 7.50e-01
epoch: 3   trainLoss: 1.0203e+00   valLoss:1.0087e+00  time: 7.56e-01
epoch: 4   trainLoss: 9.5163e-01   valLoss:9.7836e-01  time: 7.60e-01
epoch: 5   trainLoss: 8.9208e-01   valLoss:9.4693e-01  time: 7.45e-01
epoch: 6   trainLoss: 8.3510e-01   valLoss:9.1780e-01  time: 7.63e-01
epoch: 7   trainLoss: 7.8565e-01   valLoss:8.8748e-01  time: 7.68e-01
epoch: 8   trainLoss: 7.4700e-01   valLoss:8.6206e-01  time: 7.63e-01
epoch: 9   trainLoss: 7.1625e-01   valLoss:8.3598e-01  time: 7.41e-01
epoch: 10   trainLoss: 6.8914e-01   valLoss:8.1081e-01  time: 7.53e-01
epoch: 11   trainLoss: 6.6174e-01   valLoss:7.8258e-01  time: 7.37e-01
epoch: 12   trainLoss: 6.3183e-01   valLoss:7.5011e-01  time: 7.34e-01
epoch: 13   tra

alt.Chart(...)

testing on design_9



,mse,mae,mre,maxAE,mae/peak,maxAE/peak,relEAtPeak,Trained on,Tested on
0,0.000003,0.001280,0.086752,0.004402,0.078154,0.268710,0.194701,test group,design_5
1,0.000006,0.001951,0.113409,0.006593,0.058484,0.197649,0.045118,test group,design_5
2,0.000040,0.005222,0.143329,0.013249,0.091799,0.232906,0.232906,test group,design_5
3,0.000022,0.003561,0.077223,0.010990,0.036738,0.113382,0.072949,test group,design_5
4,0.000007,0.002158,0.142622,0.005293,0.152062,0.372986,0.138145,test group,design_5
...,...,...,...,...,...,...,...,...,...
3595,0.000039,0.004916,0.180882,0.014680,0.132464,0.395551,0.267883,transfer learning (20%),design_9
3596,0.000088,0.007092,0.118512,0.028958,0.044213,0.180540,0.067223,transfer learning (20%),design_9
3597,0.000011,0.002683,0.110135,0.008375,0.081173,0.253360,0.070156,transfer learning (20%),design_9
3598,0.000012,0.002730,0.085950,0.007546,0.049620,0.137142,0.133569,transfer learning (20%),design_9


In [8]:
# save test results to file
df = pd.DataFrame(resultsList)
df.to_csv(saveDir+'testResults.csv', index=False)

## 6. Plot results

In [9]:
df = pd.DataFrame(resultsList)
order = ['test group', 'all groups']
barChart = alt.Chart(df).mark_bar().encode(
    x=alt.X('Trained on:N', sort=order, title='', axis=alt.Axis(ticks=False, labels=False)),
    y=alt.Y('mean(mse):Q', scale=alt.Scale(type='log'), axis=alt.Axis(tickCount=8, format=".0e"), title='MSE'),
    color=alt.Color('Trained on:N', sort=order),
    opacity = alt.OpacityValue(0.8),
    tooltip='mean(mse):Q'
).properties(width=75, height=200)

scatter = alt.Chart(df).mark_circle(size=20).encode(
    x=alt.X('Trained on:N', title='', sort=order),
    y=alt.Y('mse:Q', scale=alt.Scale(type='log')),
    color=alt.Color('Trained on:N', sort=order),
    opacity = alt.OpacityValue(0.3),
    tooltip='mse:Q'
)

alt.layer(barChart, scatter, data=df).facet(
    column=alt.Column('Tested on:N'))

alt.FacetChart(...)